Cov2D 커널 사이즈 조절   
Conv conv pooling 반복    

In [1]:
from tensorflow.keras import initializers
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

from keras.models import Sequential,load_model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
np.random.seed(1)


train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split = 0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './images2/train',
        target_size = (300,300),
        batch_size=128,
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        seed = 42,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './images2/test',
        classes = ['food','interior', 'exterior'],
        target_size=(300,300),
        batch_size=64,
        shuffle =True,
        seed = 42,
        class_mode='categorical')

Found 36000 images belonging to 3 classes.
Found 9000 images belonging to 3 classes.


In [2]:
initializer = tf.keras.initializers.GlorotUniform()
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                 activation='relu',
                 input_shape=(300,300,3)))
model.add(Conv2D(32,(3,3), activation='relu',kernel_initializer=initializer))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), activation='relu',kernel_initializer=initializer))
model.add(Conv2D(32, (3,3), activation='relu',kernel_initializer=initializer))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(4, activation='relu',kernel_initializer=initializer))
model.add(Dense(4, activation='relu',kernel_initializer=initializer))
model.add(Dense(3, activation='softmax'))

# 3. 모델 학습과정 설정 및 요약
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
    train_generator,
    steps_per_epoch=3,
    epochs=10,
    validation_data=test_generator,
    validation_steps=3)

model.save('model-20')

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 296, 296, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 146, 146, 32)      9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 144, 144, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 72, 72, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 165888)            0

C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/10
3/3 [==============================] - 32s 11s/step - loss: 1.1044 - accuracy: 0.3802 - val_loss: 1.0979 - val_accuracy: 0.4635
Epoch 2/10
3/3 [==============================] - 33s 11s/step - loss: 1.0980 - accuracy: 0.4115 - val_loss: 1.0971 - val_accuracy: 0.4740
Epoch 3/10
3/3 [==============================] - 33s 11s/step - loss: 1.0971 - accuracy: 0.4505 - val_loss: 1.0973 - val_accuracy: 0.3906
Epoch 4/10
3/3 [==============================] - 34s 11s/step - loss: 1.0960 - accuracy: 0.4818 - val_loss: 1.0968 - val_accuracy: 0.4479
Epoch 5/10
3/3 [==============================] - 33s 11s/step - loss: 1.0957 - accuracy: 0.4740 - val_loss: 1.0944 - val_accuracy: 0.4792
Epoch 6/10
3/3 [==============================] - 33s 11s/step - loss: 1.0953 - accuracy: 0.4453 - val_loss: 1.0939 - val_accuracy: 0.4740
Epoch 7/10
3/3 [==============================] - 34s 11s/step - loss: 1.0952 - accuracy: 0.4505 - val_loss: 1.0955 - val_accuracy: 0.4115
Epoch 8/10
3/3 [===========